In [ ]:
import pandas as pd
import numpy as np

from pandas_datareader import DataReader
import yfinance as yf
import requests

import matplotlib.pyplot as plt

#### Data Extraction

In [ ]:
# data init
start_date = "2017-01-01"
end_date = "2022-01-01"
symbol = "AAPL"


# CREATING DATAFRAME
# df = DataReader(name=symbol, data_source="yahoo", start=start_date , end=end_date )

stock = yf.Ticker(symbol)
df = stock.history(start=start_date, end=end_date)


df.head()


# display ethereums ticker price from 2017 - 2022
df.tail()

#### Feature Adjustment

- average moving crossover

In [ ]:
# find moving averages
df["moving-avg-12"] = df["Close"].rolling(window=12).mean()
df["moving-avg-21"] = df["Close"].rolling(window=21).mean()


# check if rolling avrage for 12 days is more than 21 days average 
df.loc[df["moving-avg-12"] > df['moving-avg-21'] , "Signal"] = 1
df.loc[df["moving-avg-12"] <= df['moving-avg-21'] , "Signal"] = 0

# Move outputs to future dates analysis
df["Signal"] = df["Signal"].shift(1)

df

In [ ]:
# Add Returns
# NOTE: checking close price divided by yesterdays price
df["log-returns-benchmak"] = np.log(df["Close"] / df['Close'].shift(1))
df["log-returns-product-benchmak"] = df["log-returns-benchmak"].cumsum()  # 
df["product-bench"] = np.exp(df["log-returns-product-benchmak"]) - 1
df

In [ ]:
# Add Strategy Returns
# NOTE: takes open of the following day divided by the current open
df["log-returns-strategy"] = np.log(df["Open"].shift(-1) / df['Open']) * df["Signal"]
df["log-returns-product-strategy"] = df["log-returns-strategy"].cumsum()
df["product-strategy"] = np.exp(df["log-returns-product-strategy"]) - 1
df.tail()

In [ ]:
# removing NaN
df.dropna(inplace=True)

df

# Plot Results


In [ ]:
# show graph
fig = plt.Figure(figsize=(15,8))
plt.plot(df["product-bench"])
plt.plot(df["product-strategy"])
plt.show

## metrics

In [ ]:
# calculate returns
print("Benchmark Returns: " , df["product-bench"].iloc[-1])
print("Strategy Returns: " , df["product-strategy"].iloc[-1])

### calculate sharpe
- input all the log returns
- find its average
- divide the <strong>Average</strong> by the <strong>Standard Deviation</strong>

#### Per dollar of risk... How Much Reward?  
- Avewrage spent / Risk
- <strong>Standard Deviation is risk</strong>
- <strong>Average is reward</strong>


# IMPORTANT: Finding a sharpe ratio above 1.15 is doing very well. 



In [ ]:
# calculate sharpe
def calculate_sharpe(log_returns):
    days = 255
    risk_free_rate = 0.01
    sqrt_days = np.sqrt(days)
    mean = log_returns.mean() * days 
    sigma = log_returns.std() * sqrt_days
    sharpe = round( (mean - risk_free_rate) / sigma , 3)
    return sharpe

In [ ]:
# show sharpe
bench_sharpe    = calculate_sharpe(df["log-returns-benchmak"])
strategy_sharpe =  calculate_sharpe(df["log-returns-strategy"])
print("Benchmark Sharpe: " , bench_sharpe)
print("Strategy Sharpe: " , strategy_sharpe)